<a href="https://colab.research.google.com/github/MarioCastilloM/Activos_Derivados/blob/main/Options_Valuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
class Options:
  def __init__(self, s0, X, rf, sigma, div, T, n, option):
    self.s0 = s0
    self.X = X
    self.rf = rf
    self.sigma = sigma
    self.div = div
    self.T = T
    self.option = option
    self.n = n

  def priceBSM(self):
    import numpy as np
    from scipy.stats import norm
    if self.div == 0:
      d1 = (np.log((self.s0)/self.X) + (self.rf + (self.sigma**2)/2) * self.T/self.n)/(self.sigma*np.sqrt(self.T/self.n))
      d2 = d1 - self.sigma * np.sqrt(self.T/self.n)

      if self.option == "call":
        price = (self.s0)*norm.cdf(d1) - self.X*np.exp(-self.rf*self.T/self.n)*norm.cdf(d2)
      elif self.option == "put":
        price = self.X*np.exp(-self.rf*self.T/self.n)*norm.cdf(-d2) - (self.s0)*norm.cdf(-d1)
      else:
        print("ingrese opción válida")
        price = 0
    else:
      vpd = np.exp(-self.div*self.T/self.n)
      d1 = (np.log((self.s0*vpd)/self.X) + (self.rf + self.sigma**2/2)*self.T/self.n)/(self.sigma*np.sqrt(self.T/self.n))
      d2 = d1 - self.sigma * np.sqrt(self.T/self.n)

      if self.option == "call":
        price = (self.s0*vpd)*norm.cdf(d1) - self.X*np.exp(-self.rf*self.T/self.n)*norm.cdf(d2)
      elif self.option == "put":
        price = self.X*np.exp(-self.rf*self.T/self.n)*norm.cdf(-d2) - (self.s0*vpd)*norm.cdf(-d1)
      else:
        print("ingrese opción válida")
        price = 0
    return price

  def parity(self, price):
    import numpy as np
    if self.div == 0:
      if self.option == "call":
        par = price + self.X*(1+self.rf)**-self.T/self.n - self.s0
      elif self.option == "put":
        par = price - self.X*(1+self.rf)**-self.T/self.n + self.s0
      else:
        print("ingrese opción válida")
        par = 0
    else:
      vpd = np.exp(-self.div*self.T/self.n)
      if self.option == "call":
        vpd = np.exp(-self.div*self.T/self.n)
        par = price + self.X*(1+self.rf)**-self.T/self.n - self.s0*vpd
      elif self.option == "put":
        par = price - self.X*(1+self.rf)**-self.T/self.n + self.s0*vpd
      else:
        print("ingrese opción válida")
        par = 0
    return par
  def binomial_tree(self):
    import numpy as np
    import pandas as pd
    u = np.exp(self.sigma * np.sqrt(self.T/self.n))
    d = 1/u
    exp = np.exp(self.rf * self.T/self.n)
    p = (exp - d)/(u - d)
    df = pd.DataFrame(np.zeros((self.n, self.n)), columns=range(self.n))
    for i in range(self.n):
      for j in range(1, self.n):
        df.iloc[0, 0] = self.s0
        df.iloc[i, 0] = 0
        if j >= i:
          df.iloc[i, j] = self.s0 * u**(j-i) * d**i
        else:
          df.iloc[i, j] = 0
    df1 = pd.DataFrame(np.zeros(df.shape))
    if self.option == "put":
      for i in range(len(df1)-1):
        for j in range(len(df1)-1, 0-1, -1):
          if j == len(df1)-1:
            df1.iloc[i, j] = max(self.X - df.iloc[i, j], 0)
          elif i <= j:
            df1.iloc[i, j] = max(((p * df1.iloc[i, j+1] + (1-p) * df1.iloc[i+1, j+1]))/exp, self.X - df.iloc[i, j])
          else:
            df1.iloc[i, j] = 0
    elif self.option == "call":
      for i in range(len(df1)-1):
        for j in range(len(df1)-1, 0-1, -1):
          if j == len(df1)-1:
            df1.iloc[i, j] = max(df.iloc[i, j] - self.X, 0)
          elif i <= j:
            df1.iloc[i, j] = max(((p * df1.iloc[i, j+1] + (1-p) * df1.iloc[i+1, j+1]))/exp, df.iloc[i, j] - self.X)
          else:
            df1.iloc[i, j] = 0
    else:
        print("option type is not valid")
        par = 0
    return df1.iloc[0, 0]

  def monte_carlo_ln(self, J):
    import numpy as np
    e = np.random.normal(0, 1, J)
    op = np.zeros(J)
    for i in range(J):
      st = self.s0 * np.exp((self.rf - ((self.sigma**2) / 2)) * self.T/self.n + self.sigma * e[i] * np.sqrt(self.T/self.n))
      if self.option == "call":
        op[i] = max(st - self.X, 0) * np.exp(-self.rf * self.T/self.n)
      elif self.option == "put":
        op[i] = max(self.X - st, 0) * np.exp(-self.rf * self.T/self.n)
      else:
        op[i] = 0
    price = np.mean(op)
    return price
  def monte_carlo_bt(self, J):
    import numpy as np
    delta_t = self.T/self.n
    u = np.exp(self.sigma * np.sqrt(delta_t))
    d = 1/u
    p = (np.exp(self.rf * delta_t) - d)/(u - d)
    op = np.zeros(J)
    for i in range(J):
      probs = np.random.uniform(0, 1, self.n)
      delt = len(probs[probs < (1-p)])
      st = self.s0 * (d**delt) * (u**(self.n - delt))
      if self.option == "call":
        op[i] = max(st - self.X, 0) * np.exp(-self.rf * self.T)
      elif self.option == "put":
        op[i] = max(self.X - st, 0) * np.exp(-self.rf * self.T)
      else:
        op[i] = 0
    price = np.mean(op)
    return price

In [55]:
p1 = Options(s0 = 50, X = 52, rf = 0.05, sigma=0.3, div=0, T = 2, n = 500, option="put")
pp1 = p1.priceBSM()
print(pp1)
# parp1 = p1.parity(pp1)
# print(parp1)
binomial = p1.binomial_tree()
print(binomial)
MC = p1.monte_carlo_ln(J = 1000)
print(MC)
# MCBT = p1.monte_carlo_bt(J = 1000)
# print(MCBT)

1.9966499333931296
2.0
1.997677234937928


In [52]:
# arbol
import numpy as np
n = 500
delta_t = 2/n
u = np.exp(0.3 * np.sqrt(delta_t))
d = 1/u
p = (np.exp(0.05 * delta_t) - d)/(u - d)
J = 30000
op = np.zeros(J)
for i in range(J):
  probs = np.random.uniform(0, 1, n)
  delt = len(probs[probs < (1-p)])
  st = 50 * (d**delt) * (u**(n - delt))
  op[i] = max(52 - st, 0) * np.exp(-0.05 * T)
price = np.mean(op)

In [53]:
price

6.770686333791812

In [3]:
delt

244

In [4]:
st

62.78442272718567

In [6]:
op

array([ 0.        ,  2.00040004,  0.        , ...,  0.        ,
       21.47435905,  0.        ])

In [45]:
from scipy.stats import norm
s0 = 50
X = 52
rf = 0.05
sigma = 0.3
T = 2
n = 500
deltaT = T/n

d1 = (np.log(s0/X) + (rf + (sigma**2)/2)*T)/(sigma*np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)

c = X*np.exp(-rf*T)*norm.cdf(-d2) - (s0)*norm.cdf(-d1)

print(f'Los valores relevantes de la valoración de la opción son:\n\
Dados los valores anteriores, al aplicar BSM el valor de la call es: {round(c, 2)}')

Los valores relevantes de la valoración de la opción son:
Dados los valores anteriores, al aplicar BSM el valor de la call es: 6.76


In [25]:
deltaT

0.004

In [26]:
u

1.0191548098015244

In [27]:
d

0.9812052009986052

In [28]:
1-p

0.49947260059062537

In [46]:

import numpy as np

from scipy.stats import norm
s0 = 50
X = 52
rf = 0.05
sigma = 0.3
T = 2
n = 500
deltaT = T/n

J = 30000
e = np.random.normal(0, 1, J)
op = np.zeros(J)
for i in range(J):
  st = s0 * np.exp((rf - ((sigma**2) / 2)) * T + sigma * e[i] * np.sqrt(T))
  op[i] = max(X - st, 0) * np.exp(-rf * T)

In [47]:
price = np.mean(op)

In [48]:
price

6.73938957024994

In [56]:
import numpy as np
import pandas as pd

s0 = 50
X = 52
rf = 0.05
sigma = 0.3
T = 2
n = 500
deltaT = T/n

u = np.exp(sigma * np.sqrt(T/n))
d = 1/u
exp = np.exp(rf * T/n)
p = (exp - d)/(u - d)
df = pd.DataFrame(np.zeros((n, n)), columns=range(n))
for i in range(n):
  for j in range(1, n):
    df.iloc[0, 0] = s0
    df.iloc[i, 0] = 0
    if j >= i:
      df.iloc[i, j] = s0 * u**(j-i) * d**i
    else:
      df.iloc[i, j] = 0
df1 = pd.DataFrame(np.zeros(df.shape))
for i in range(len(df1)-1):
  for j in range(len(df1)-1, 0-1, -1):
    if j == len(df1)-1:
      df1.iloc[i, j] = max(X - df.iloc[i, j], 0)
    elif i <= j:
      df1.iloc[i, j] = max(((p * df1.iloc[i, j+1] + (1-p) * df1.iloc[i+1, j+1]))/exp, X - df.iloc[i, j])
    else:
      df1.iloc[i, j] = 0

df1.iloc[0, 0]

2.0

In [61]:
df.iloc[499, 499]

0.00386483492097607

In [58]:
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,2.0,1.04226,0.066174,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
1,0.0,2.93974,2.000000,1.042260,0.066174,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.00000,3.861818,2.939740,2.000000,1.042260,0.066174,0.00000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.00000,0.000000,4.766565,3.861818,2.939740,2.000000,1.04226,0.066174,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.00000,0.000000,0.000000,5.654308,4.766565,3.861818,2.93974,2.000000,1.04226,0.066174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.99583,51.995751,51.995669,51.995586,51.995502
496,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,51.995909,51.995830,51.995751,51.995669
497,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,51.995986,51.995909,51.995830
498,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,